In [22]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def compute_derivs(g, m1, r1, m2, r2, X):
    a1 = X[0]
    a2 = X[1]
    da1 = X[2]
    da2 = X[3]
    
    k = m2*r1*r2*np.cos(a1-a2)
    
    l = np.array([
        [k,             r2*r2*m2],
        [r1*r1*(m1+m2), k       ]
        
    ])
    
    c = np.array([
        [(m2*r1*r2*da1*da1*np.sin(a1-a2))-(m2*g*r2*np.sin(a2))],
        [-(m2*r1*r2*da2*da2*np.sin(a1-a2))-(g*r1*np.sin(a1)*(m1+m2))]
    ])
    
    x = np.dot(np.linalg.inv(l), c)
#     print(x[0])
    
    return np.array([da1, da2, x[0][0], x[1][0]])

def do_rk4(a10, a20, da10, da20, t, h, g, m1, m2, r1, r2):
    X = np.zeros((len(t), 4))
    
    X[0][0] = a10
    X[0][1] = a20
    X[0][2] = da10
    X[0][3] = da20
    
    for i in range(1, len(t)):
        k1 = compute_derivs(g, m1, r1, m2, r2, X[i-1])
#         print(k1)
        k2 = compute_derivs(g, m1, r1, m2, r2, X[i-1]+((h*k1)/2))
        k3 = compute_derivs(g, m1, r1, m2, r2, X[i-1]+((h*k2)/2))
        k4 = compute_derivs(g, m1, r1, m2, r2, X[i-1]+(h*k3))
        X[i] = X[i-1]+(k1+k2+k2+k3+k3+k4)*(h/6)
        
    return X

def compute_total_energy(X, m1, m2, r1, r2, g):
    x = X.T
    v2 = np.multiply(x[0]**2, x[3]**2)+(x[2]**2)
    v12 = r1*r1*(x[2]**2)
    v22 = (r1*r1*(x[2]**2))+(r2*r2*(x[3]**2))+(2*r1*r2*np.multiply(np.multiply(x[2], x[3]), np.cos(x[0]-x[1])))
    ke = 0.5*((m1*v12)+(m2*v22))
    
    pe = (-g)*((m1*r1*np.cos(x[0])) + (m2*((r1*np.cos(x[0]))+(r2*np.cos(x[1])))))
    return ke,pe

In [66]:
g = 9.81
m1 = 1
m2 = m1
r1 = 0.5
r2 = 1

a1 = -3
a2 = 3
da1 = 0
da2 = 0

h = 0.0005
t = np.arange(0, 40+h, h)

V = do_rk4(a1, a2, da1, da2, t, h, g, m1, m2, r1, r2)
ke, pe = compute_total_energy(V, m1, m2, r1, r2, g)
te = ke+pe

In [67]:
%matplotlib qt
plt.figure()
plt.plot(t, V.T[0])
plt.show()
plt.figure()
plt.plot(t, V.T[1])
plt.show()
plt.figure()
plt.plot(t, te)
plt.show()

In [68]:
X1 = r1*np.sin(V.T[0])
Y1 = -r1*np.cos(V.T[0])

X2 = X1+r2*np.sin(V.T[1])
Y2 = Y1-r2*np.cos(V.T[1])

f = 20

fig, ax = plt.subplots()
ax.set_xlim(min(min(X2), min(X1))-1, max(max(X2), max(X1))+1)
ax.set_ylim(min(min(Y2), min(Y1))-1, max(max(Y2), max(Y1))+1)
# ax.set_ylim(min(Y2)-5, max(Y2)+1)
ax.scatter(0, 0, c='black', s=80, label='Fixed point (0, 0)')
scat_1 = ax.scatter([], [], s=[], c='blue')
scat_2 = ax.scatter([], [], s=[], c='red')
trail_line_1, = ax.plot([], [], 'b--', linewidth=1, alpha=0.6)
trail_line_2, = ax.plot([], [], 'r--', linewidth=1, alpha=0.6)
fixed_line_1, = ax.plot([], [], color='orange', linewidth=1, label='Fixed to Point')
fixed_line_2, = ax.plot([], [], color='orange', linewidth=1, label='Fixed to Point')

coord_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
a1_text = ax.text(0.02, 0.88, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
a2_text = ax.text(0.02, 0.81, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
energy_text = ax.text(0.02, 0.73, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
time_text = ax.text(0.8, 0.95, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
trail_x1 = []
trail_y1 = []
trail_x2 = []
trail_y2 = []

def init():
    trail_x1.clear()
    trail_y1.clear()
    trail_x2.clear()
    trail_y2.clear()
    trail_line_1.set_data([], [])
    trail_line_2.set_data([], [])
    
    scat_1.set_offsets([[X2[0], Y2[0]]])
    scat_1.set_sizes([10])
    scat_2.set_offsets([[X1[0], Y1[0]]])
    scat_2.set_sizes([10])
    coord_text.set_text(f'Coordinates: ({X2[0]:.2f}, {Y2[0]:.2f})')
    a1_text.set_text(f'Theta1: ({V.T[0][0]:.2f})')
    a2_text.set_text(f'Theta2: ({V.T[1][0]:.2f})')
    energy_text.set_text(f'Energy: ({te[0]:.2f})')
    time_text.set_text(f'Time: ({t[0]:.2f})')
    fixed_line_1.set_data([], [])
    fixed_line_2.set_data([], [])
    return scat, coord_text, a1_text, a2_text, energy_text, time_text, fixed_line

def update(frame):
    x1, y1 = X1[f*frame], Y1[f*frame]
    x2, y2 = X2[f*frame], Y2[f*frame]
    trail_x1.append(x1)
    trail_y1.append(y1)
    trail_x2.append(x2)
    trail_y2.append(y2)
    
    scat_1.set_offsets([[x1, y1]])
    scat_1.set_sizes([10])
    scat_2.set_offsets([[x2, y2]])
    scat_2.set_sizes([10])
    trail_line_1.set_data(trail_x1, trail_y1)
    trail_line_2.set_data(trail_x2, trail_y2)
    fixed_line_1.set_data([0, x1], [0, y1])
    fixed_line_2.set_data([x2, x1], [y2, y1])
    coord_text.set_text(f'Coordinates: ({x2:.2f}, {y2:.2f})')
    a1_text.set_text(f'Theta1: ({V.T[0][f*frame]:.2f})')
    a2_text.set_text(f'Theta2: ({V.T[1][f*frame]:.2f})')
    energy_text.set_text(f'Energy: ({te[f*frame]:.2f})')
    time_text.set_text(f'Time: ({t[f*frame]:.2f})')
    return scat, trail_x1, trail_x2, coord_text, a1_text, a2_text, energy_text, time_text, fixed_line

ani = animation.FuncAnimation(fig, update, frames=(len(X2)//f),
                              init_func=init, interval=1)

plt.show()
